# Lecture5 「マンガと学ぶデータビジュアライゼーション」

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install ijson


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install tqdm


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import glob
import ijson
import json
import numpy as np
import os
import pandas as pd
from pprint import pprint
from tqdm import tqdm_notebook as tqdm
import zipfile

In [5]:
DIR_IN = 'madb/data/json-ld'
DIR_TMP = 'data/preprocess/tmp'
DIR_OUT = 'data/preprocess/out'

In [6]:
FNS_CM = [
    'cm102',
    'cm105',
    'cm106',
]

In [7]:
# 分析対象とする雑誌名
MCNAMES = [
    '週刊少年ジャンプ',
    '週刊少年マガジン', 
    '週刊少年サンデー',
    '週刊少年チャンピオン',
]

In [8]:
COLS_CM105 = [
    'identifier',
    'label',
    'name',
]

In [9]:
# cm102, genre=='雑誌巻号'
COLS_MIS = {
    'identifier': 'miid',
    'label': 'miname',
    'datePublished': 'datePublished',
    'isPartOf': 'mcid',
    'issueNumber': 'issueNumber',
    'numberOfPages': 'numberOfPages',
    'publisher': 'publisher',
    'volumeNumber': 'volumeNumber',
    'price': 'price',
    'editor': 'editor',
}

In [10]:
# cm102, genre=='マンガ作品'
COLS_EPS = {
    'relatedCollection': 'cid',
    'creator': 'creator',
    'note': 'note',
    'alternativeHeadline': 'epname',
    'pageStart': 'pageStart',
    'pageEnd': 'pageEnd',
    'isPartOf': 'miid',
}

In [11]:
# cm106
COLS_CS = {
    'identifier': 'cid', 
    'name': 'cname'
}

In [12]:
# 最終的に出力するカラム
COLS_OUT = [
    'mcname', 'miid', 'miname', 'cid', 'cname', 'epname',
    'creator', 'pageStart', 'pageEnd', 'numberOfPages',
    'datePublished', 'price', 'publisher', 'editor',
]

In [13]:
# 許容するpageEnd，pageStartの最大値
MAX_PAGES = 1000

In [14]:
def read_json(path):
    """
    jsonファイルを辞書として読み込む関数．

    Params:
        path (str): 読込対象ファイルパス
    Returns:
        dict: 辞書
    """
    with open(path, 'r', encoding="utf-8") as f:
        dct = json.load(f)

    return dct

In [15]:
def read_json2(path):
    """
    jsonファイルを辞書として読み込む関数．

    Params:
        path (str): 読込対象ファイルパス
    Returns:
        dict: 辞書
    """
    with open(path, 'r') as f:
        dct = json.load(f)

    return dct

In [16]:
def save_json(path, dct):
    """
    辞書をjson形式で保存する関数．

    Params:
        path (str): jsonファイルの保存先
        dct (dict): 保存対象辞書
    """
    with open(path, 'w') as f:
        json.dump(dct, f, ensure_ascii=False, indent=4)

In [18]:
def read_json_w_filters(path, items, filters):
    """itemsのうち，filtersの条件を満たすもののみを抽出
    path: jsonファイルのパス
    items: jsonファイル中でitemsを取得するキー
    filters: dict形式．item[key] in valueで条件づけする想定
    """
    out = []
    with open(path, 'r', encoding="utf-8") as f:
        parse = ijson.items(f, items)
        for item in parse:
            # filtersの条件をすべて満足するもの以外はbreak
            for k, v in filters.items():
                if k not in item.keys():
                    break
                if item[k] not in v:
                    break
            else:
                # breakしなかった場合はoutに追加
                out.append(item)
    return out

In [19]:
def try_mkdirs(path) -> None:
    """mkdirsにtry"""
    try:
        os.makedirs(path)
    except FileExistsError as e:
        pass

In [20]:
try_mkdirs(DIR_TMP)
try_mkdirs(DIR_OUT)

In [21]:
DIR_OUT

'data/preprocess/out'

In [22]:
ps_cm = glob.glob(f'{DIR_IN}/*_cm*')

In [23]:
ps_cm

['madb/data/json-ld\\metadata_cm-col_cm105_json.zip',
 'madb/data/json-ld\\metadata_cm-col_cm106_json.zip',
 'madb/data/json-ld\\metadata_cm-item_cm102_json.zip']

In [24]:
FNS_CM

['cm102', 'cm105', 'cm106']

In [22]:
#%pip install --upgrade jupyter ipywidgets
#%jupyter nbextension enable --py widgetsnbextension # removed !pip on the recommendation of a comment.


In [25]:
for p_from in tqdm(ps_cm):
    p_to = p_from.replace(DIR_IN, DIR_TMP).replace('.zip', '')
    
    with zipfile.ZipFile(p_from) as z:
        z.extractall(p_to)

  0%|          | 0/3 [00:00<?, ?it/s]

In [26]:
ps_cm = {cm: glob.glob(f'{DIR_TMP}/*{cm}*/*/*') for cm in FNS_CM}

In [27]:
FNS_CM

['cm102', 'cm105', 'cm106']

In [28]:
glob.glob(f'{DIR_TMP}/*cm105*/*/*') 

['data/preprocess/tmp\\metadata_cm-col_cm105_json\\metadata_cm-col_cm105_json\\metadata_cm-col_cm105_00001.json']

In [29]:
pprint(ps_cm)

{'cm102': ['data/preprocess/tmp\\metadata_cm-item_cm102_json\\metadata_cm-item_cm102_json\\metadata_cm-item_cm102_00001.json',
           'data/preprocess/tmp\\metadata_cm-item_cm102_json\\metadata_cm-item_cm102_json\\metadata_cm-item_cm102_00002.json'],
 'cm105': ['data/preprocess/tmp\\metadata_cm-col_cm105_json\\metadata_cm-col_cm105_json\\metadata_cm-col_cm105_00001.json'],
 'cm106': ['data/preprocess/tmp\\metadata_cm-col_cm106_json\\metadata_cm-col_cm106_json\\metadata_cm-col_cm106_00001.json']}


In [30]:
ps_cm={'cm102': ['data/preprocess/tmp\\metadata_cm-item_cm102_json\\metadata_cm-item_cm102_json\\metadata_cm-col_cm105_00001.json'],
 'cm105': ['data/preprocess/tmp\\metadata_cm-col_cm105_json\\metadata_cm-col_cm105_json\\metadata_cm-col_cm105_00001.json'],
 'cm106': ['data/preprocess/tmp\\metadata_cm-col_cm106_json\\metadata_cm-col_cm106_json\\metadata_cm-col_cm106_00001.json']}

cm105
漫画雑誌に関するデータを整形し，分析対象のIDを特定

In [31]:
def format_magazine_name(name):
    """nameからpublished_nameを取得"""
    for x in name:
        if type(x) is str:
            return x
    raise Exception(f'No magazine name in {name}!')

In [32]:
ps_cm['cm105'][0]

'data/preprocess/tmp\\metadata_cm-col_cm105_json\\metadata_cm-col_cm105_json\\metadata_cm-col_cm105_00001.json'

In [33]:
cm105 = read_json(ps_cm['cm105'][0])
df_cm105 = pd.DataFrame(cm105['@graph'])[COLS_CM105]

In [34]:
# 雑誌名を取得
df_cm105['mcname'] = df_cm105['name'].apply(
    lambda x: format_magazine_name(x))

In [35]:
# mcnameで抽出
df_cm105[df_cm105['mcname'].isin(MCNAMES)].T

,148,1449,1828,2569
identifier,C117607,C119033,C119459,C120282
label,週刊少年サンデー,週刊少年マガジン,週刊少年ジャンプ,週刊少年チャンピオン
name,"[{'@language': 'ja-hrkt', '@value': 'シュウカンショウネ...","[週刊少年マガジン, {'@language': 'ja-hrkt', '@value': ...","[週刊少年ジャンプ, {'@language': 'ja-hrkt', '@value': ...","[{'@language': 'ja-hrkt', '@value': 'シュウカンショウネ..."
mcname,週刊少年サンデー,週刊少年マガジン,週刊少年ジャンプ,週刊少年チャンピオン


In [36]:
# 雑誌ID:雑誌名
mcid2mcname = \
    df_cm105[df_cm105['mcname'].isin(MCNAMES)].\
    groupby('identifier')['mcname'].first().to_dict()

In [37]:
# 保存
save_json(os.path.join(DIR_TMP, 'mcid2mcname.json'), mcid2mcname)

cm102
雑誌巻号およびマンガ作品に関するデータを整形し，一次保存．

In [38]:
def format_cols(df, cols_rename):
    """cols_renameのcolのみを抽出し，renmae"""
    df_new = df.copy()
    df_new = df_new[cols_rename.keys()]
    df_new = df_new.rename(columns=cols_rename)
    return df_new

In [39]:
def get_items_by_genre(graph, genre):
    """graphから所定のgenreのアイテム群を取得"""
    items = [
        x for x in graph 
        if 'genre' in x.keys() and x['genre'] == genre]
    return items

In [40]:
def get_id_from_url(url):
    """url表記から末尾のidを取得"""
    if url is np.nan:
        return None
    else:
        return url.split('/')[-1]

In [41]:
def format_nop(numberOfPages):
    """numberOfPagesからpやPを除外"""
    nop = numberOfPages
    if nop is np.nan:
        return None
    elif nop == '1冊' or nop == '1サツ':
        # M577294, 週刊少年サンデー 2010年 表示号数17
        # nop = '1冊'
        return None
    else:
        return int(nop.replace('p', '').replace('P', ''))

In [42]:
def format_price(price):
    """priceを整形"""
    if price is np.nan:
        return None
    elif price == 'JUMPガラガラウなかも':
        # M544740, 週刊少年ジャンプ 1971年 表示号数47
        # price = 'JUMPガラガラウなかも'
        return None
    elif price == '238p':
        # M542801, 週刊少年ジャンプ 2010年 表示号数42
        # price = '238p'
        return 238
    else:
        price_new = price.replace('円', '').replace('+税', '')
        return int(price_new)

In [57]:
def format_creator(creator):
    """creatorから著者名を取得"""
    if creator is np.nan:
        return None
    for x in creator:
        if type(x) is str:
            return x
    raise Exception('No creator name!')

In [45]:
def create_df_mis(path, mcids):
    """pathとmcidsからdf_misを構築"""    
    filters = {
        'genre': ['雑誌巻号'],
#        'genre': ['マンガ雑誌単号'],
        'isPartOf': [
            f'https://mediaarts-db.bunka.go.jp/id/{mcid}' for mcid in mcids],
            #f'https://mediaarts-db.artmuseums.go.jp/id/{mcid}' for mcid in mcids],
    }
    mis = read_json_w_filters(path, '@graph.item', filters)
    df_mis = pd.DataFrame(mis)
    
    # 列を整理
    df_mis = format_cols(df_mis, COLS_MIS)
    # mcidを取得
    df_mis['mcid'] = df_mis['mcid'].apply(
        lambda x: get_id_from_url(x))
    # datePublishedでソート
    df_mis['datePublished'] = pd.to_datetime(df_mis['datePublished'])
    df_mis  = df_mis.sort_values('datePublished', ignore_index=True)
    # numberOfPagesを整形
    df_mis['numberOfPages'] = df_mis['numberOfPages'].apply(
        lambda x: format_nop(x))
    # priceを整形
    df_mis['price'] = df_mis['price'].apply(
        lambda x: format_price(x))
    return df_mis

In [59]:
ps_cm['cm102']

['data/preprocess/tmp\\metadata_cm-item_cm102_json\\metadata_cm-item_cm102_json\\metadata_cm-col_cm105_00001.json']

In [58]:
###y9gey9ge
for i, p in tqdm(enumerate(ps_cm['cm102'])):
    print(i,p)

0it [00:00, ?it/s]

0 data/preprocess/tmp\metadata_cm-item_cm102_json\metadata_cm-item_cm102_json\metadata_cm-col_cm105_00001.json


In [60]:
mcids

['C117607', 'C119033', 'C119459', 'C120282']

In [61]:
i=0
p='data/preprocess/tmp\metadata_cm-item_cm102_json\metadata_cm-item_cm102_00001.json'
p='data/preprocess/tmp/metadata_cm-item_cm102_json/metadata_cm-item_cm102_00001.json'
p='data/preprocess/tmp/metadata_cm-item_cm102_json/metadata_cm-item_cm102_json/metadata_cm-item_cm102_00001.json'

mcids = list(mcid2mcname.keys())
#df_mis = create_df_mis(p, mcids)
"""pathとmcidsからdf_misを構築"""    
filters = {
    'genre': ['マンガ雑誌単号'],
    'isPartOf': [
        f'https://mediaarts-db.bunka.go.jp/id/{mcid}' for mcid in mcids],
        #f'https://mediaarts-db.artmuseums.go.jp/id/{mcid}' for mcid in mcids],
}
mis = read_json_w_filters(p, '@graph.item', filters)
df_mis = pd.DataFrame(mis)
miids = set(df_mis['miid'].unique())

KeyError: 'miid'

In [51]:
mis

[]

In [45]:
miids

NameError: name 'miids' is not defined

In [ ]:
mcids


['C117607', 'C119033', 'C119459', 'C120282']

['C117607',週刊少年サンデー、マンガ雑誌
 'C119033', 週刊少年マガジン
 'C119459',週刊少年ジャンプ
  'C120282'週刊少年チャンピオン
  ]

In [ ]:
def create_df_eps(path, miids):
    """pathとmiidsからdf_epsを構築"""
    filters = {
        'genre': ['マンガ作品'],
        'isPartOf': [
            f'https://mediaarts-db.bunka.go.jp/id/{miid}' for miid in miids],
            #f'https://mediaarts-db.artmuseums.go.jp/id/{miid}' for miid in miids],
    }
    eps = read_json_w_filters(path, '@graph.item', filters)
    df_eps = pd.DataFrame(eps)
    
    # 列を整形
    df_eps = format_cols(df_eps, COLS_EPS)
    # url表記から各idを取得
    df_eps['cid'] = df_eps['cid'].apply(lambda x: get_id_from_url(x))
    df_eps['miid'] = df_eps['miid'].apply(lambda x: get_id_from_url(x))
    # 著者名を取得
    df_eps['creator'] = df_eps['creator'].apply(
        lambda x: format_creator(x))
    return df_eps

In [ ]:
for i, p in tqdm(enumerate(ps_cm['cm102'])):
    mcids = list(mcid2mcname.keys())
    df_mis = create_df_mis(p, mcids)
    # 雑誌巻号のmiidを取得し，epsの抽出に利用
    miids = set(df_mis['miid'].unique())
    df_eps = create_df_eps(p, miids)
    
    # 保存
    fn_mis = os.path.join(DIR_TMP, f'mis_{i+1:05}.csv')
    fn_eps = os.path.join(DIR_TMP, f'eps_{i+1:05}.csv')
    df_mis.to_csv(fn_mis, index=False)
    df_eps.to_csv(fn_eps, index=False)

0it [00:00, ?it/s]

cm106
掲載作品に関するデータを整形し，一次保存

In [ ]:
def format_cname(cname):
    """cnameから著者名を取得"""
    if cname is np.nan:
        return None
    for x in cname:
        if type(x) is str:
            return x
    raise Exception('No comic name!')

In [ ]:
cm106 = read_json(ps_cm['cm106'][0])

In [ ]:
# 雑誌掲載ジャンルのアイテムを抽出
cs = get_items_by_genre(cm106['@graph'], '雑誌掲載')
# DataFrame化
df_cs = pd.DataFrame(cs)
# カラムを整理
df_cs = format_cols(df_cs, COLS_CS)
# cnameを整形
df_cs['cname'] = df_cs['cname'].apply(
    lambda x: format_cname(x))

In [ ]:
# 保存
df_cs.to_csv(os.path.join(DIR_TMP, 'cs.csv'), index=False)

加工
結合

In [ ]:
def read_and_concat_csvs(pathes):
    """pathesのcsvを順番に呼び出し，concat"""
    df_all = pd.DataFrame()
    for p in pathes:
        df = pd.read_csv(p)
        df_all = pd.concat([df_all, df], ignore_index=True)
    return df_all

In [ ]:
def sort_date(df, col_date):
    """dfをcol_dateでソート"""
    df_new = df.copy()
    df_new[col_date] = pd.to_datetime(df_new[col_date])
    df_new = df_new.sort_values(col_date, ignore_index=True)
    return df_new

In [ ]:
# 各ファイルのパスを抽出
ps_mis = glob.glob(f'{DIR_TMP}/mis*.csv')
ps_eps = glob.glob(f'{DIR_TMP}/eps*.csv')
ps_cs = glob.glob(f'{DIR_TMP}/cs*.csv')

In [ ]:
# データの読み出し
df_mis = read_and_concat_csvs(ps_mis)
df_eps = read_and_concat_csvs(ps_eps)
df_cs = read_and_concat_csvs(ps_cs)
mcid2mcname = read_json2(os.path.join(DIR_TMP, 'mcid2mcname.json'))

In [ ]:
# 結合
df_all = pd.merge(df_eps, df_cs, on='cid', how='left')
df_all = pd.merge(df_all, df_mis, on='miid', how='left')
df_all['mcname'] = df_all['mcid'].apply(
    lambda x: mcid2mcname[x])

In [ ]:
# 必要な列のみ抽出
df_all = df_all[COLS_OUT]

In [ ]:
# ソート
df_all['datePublished'] = pd.to_datetime(df_all['datePublished'])
df_all = df_all.sort_values(['datePublished', 'pageStart'], ignore_index=True)

各雑誌のdatePublishedを統一

In [ ]:
df_all.groupby('mcname')['datePublished'].min()

mcname
週刊少年サンデー     1959-04-05
週刊少年ジャンプ     1969-11-03
週刊少年チャンピオン   1970-07-27
週刊少年マガジン     1959-03-26
Name: datePublished, dtype: datetime64[ns]

In [ ]:
df_all.groupby('mcname')['datePublished'].max()

mcname
週刊少年サンデー     2017-07-12
週刊少年ジャンプ     2017-07-31
週刊少年チャンピオン   2017-07-13
週刊少年マガジン     2017-07-26
Name: datePublished, dtype: datetime64[ns]

In [ ]:
# 全雑誌のうちDBに存在する期間が最も短いものに合わせる
date_min = df_all.groupby('mcname')['datePublished'].min().max()
date_max = df_all.groupby('mcname')['datePublished'].max().min()
df_all = df_all[
    (df_all['datePublished']>=date_min)&\
    (df_all['datePublished']<=date_max)].reset_index(drop=True)

In [ ]:
df_all.groupby('mcname')['datePublished'].min()

mcname
週刊少年サンデー     1970-08-02
週刊少年ジャンプ     1970-07-27
週刊少年チャンピオン   1970-07-27
週刊少年マガジン     1970-08-02
Name: datePublished, dtype: datetime64[ns]

In [ ]:
df_all.groupby('mcname')['datePublished'].max()

mcname
週刊少年サンデー     2017-07-12
週刊少年ジャンプ     2017-07-10
週刊少年チャンピオン   2017-07-06
週刊少年マガジン     2017-07-12
Name: datePublished, dtype: datetime64[ns]

In [ ]:
df_all.value_counts('mcname')

mcname
週刊少年サンデー      46179
週刊少年チャンピオン    45426
週刊少年マガジン      45318
週刊少年ジャンプ      43045
dtype: int64

適切なpageStart/pageEndを持つ行のみ抽出

In [ ]:
# pageStartがpageEndより小さい値であること
asst_ps_pe = df_all['pageStart'] <= df_all['pageEnd']
# pageEndがMAX_PAGES内であること
asst_pe = df_all['pageEnd'] <= MAX_PAGES

In [ ]:
# 抽出後のデータ
df_new = df_all[
    (asst_ps_pe&asst_pe)].reset_index(drop=True)
# 除外したデータ
df_drop = df_all[
    ~(asst_ps_pe&asst_pe)].reset_index(drop=True)

In [ ]:
# 検証
assert df_all.shape[0] == df_new.shape[0] + df_drop.shape[0]

In [ ]:
df_drop

,mcname,miid,miname,cid,cname,epname,creator,pageStart,pageEnd,numberOfPages,datePublished,price,publisher,editor
0,週刊少年ジャンプ,M544705,週刊少年ジャンプ 1972年 表示号数30,C88437,くそ坊主ガン鉄,母よどこにの巻,手無功,220.0,219.0,308.0,1972-07-17,100.0,集英社 ∥ シュウエイシャ,長野規
1,週刊少年ジャンプ,M544697,週刊少年ジャンプ 1972年 表示号数39,C88021,あらし!三匹,水上スキーの巻,池沢さとし,290.0,289.0,316.0,1972-09-11,100.0,集英社,長野規
2,週刊少年ジャンプ,M544696,週刊少年ジャンプ 1972年 表示号数40,C89703,燃えて走れ!,NaN,村上もとか,107.0,106.0,316.0,1972-09-18,100.0,集英社 ∥ シュウエイシャ,長野規
3,週刊少年サンデー,M579126,週刊少年サンデー 1973年 表示号数44,C92856,ダメおやじ,NaN,古谷三敏,189.0,150.0,334.0,1973-10-21,120.0,小学館 ∥ ショウガクカン,井上敬三
4,週刊少年ジャンプ,M544561,週刊少年ジャンプ 1975年 表示号数20,C88276,学ちゃんとメリー,NaN,大画としゆき,52.0,1.0,268.0,1975-05-19,130.0,集英社 ∥ シュウエイシャ,中野祐介
5,週刊少年サンデー,M578825,週刊少年サンデー 1979年 表示号数37,C92118,あばれ大海,にわかライダーの巻,六田登,265.0,232.0,380.0,1979-09-09,170.0,小学館,田中一喜
6,週刊少年ジャンプ,M544336,週刊少年ジャンプ 1979年 表示号数44,C89510,ふたりのダービー,NaN,田中つかさ,5.0,0.0,344.0,1979-10-29,150.0,集英社 ∥ シュウエイシャ,西村繁男
7,週刊少年ジャンプ,M544335,週刊少年ジャンプ 1979年 表示号数45,C89510,ふたりのダービー,NaN,田中つかさ,67.0,NaN,336.0,1979-11-05,150.0,集英社 ∥ シュウエイシャ,西村繁男
8,週刊少年ジャンプ,M544334,週刊少年ジャンプ 1979年 表示号数46,C89510,ふたりのダービー,NaN,田中つかさ,165.0,NaN,336.0,1979-11-12,150.0,集英社 ∥ シュウエイシャ,西村繁男
9,週刊少年ジャンプ,M544331,週刊少年ジャンプ 1979年 表示号数48,C89510,ふたりのダービー,NaN,田中つかさ,235.0,0.0,382.0,1979-11-26,170.0,集英社 ∥ シュウエイシャ,西村繁男


In [ ]:
# 全データ
df_new.to_csv(
    os.path.join(DIR_OUT, 'episodes.csv'), index=False,
    encoding='utf_8_sig')
# 除外したデータ
df_new.to_csv(os.path.join(DIR_OUT, 'droped_episodes.csv'), index=False)